<a href="https://colab.research.google.com/github/natdebandi/text_mining_unrn/blob/main/topic_modelling_ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TEXT MINING

UNRN

Especialización en ciencias de datos aplicada

Natalia Debandi

ndebandi@unrn.edu.ar

**CLASE 4**

Aprendizaje no supervisado - Topic modelling

Ejercicio

In [2]:
!pip install gensim
#Si les llega a falla la siguiente linea ejecuten esto y reinicien la sesion. A veces falla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [14]:
!pip install feedparser beautifulsoup4 requests nltk gensim matplotlib wordcloud fake-useragent pyldavis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.6 MB/s eta 0:00:00


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import feedparser
from bs4 import BeautifulSoup
import requests
import re
from fake_useragent import UserAgent
import time
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Configuración inicial
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Voy a armar un dataset para hacer LDA a partir de google news

In [56]:
def fetch_article_content(url, timeout=10):
    """Improved content extraction with better error handling"""
    try:
        headers = {'User-Agent': ua.random}
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises exception for 4XX/5XX errors
        except requests.exceptions.RequestException as e:
            print(f"Request failed for {url}: {e}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Try different strategies to get article content
        strategies = [
            lambda: soup.find('article'),
            lambda: soup.find(class_=re.compile('article|content|body|post', re.I)),
            lambda: soup.find(itemprop='articleBody'),
            lambda: soup.find('main'),
            lambda: soup.find('body')
        ]

        article_content = None
        for strategy in strategies:
            article_content = strategy()
            if article_content:
                break

        if not article_content:
            return None

        # Clean up the content
        for element in article_content(['script', 'style', 'iframe', 'nav', 'footer', 'aside', 'header']):
            element.decompose()

        text = ' '.join([p.get_text() for p in article_content.find_all(['p', 'h1', 'h2', 'h3'])])
        text = re.sub(r'\s+', ' ', text).strip()
        return text[:10000] if text else None

    except Exception as e:
       # print(f"Error processing {url}: {e}")
        return None


In [57]:
def extraer_noticias_completas(query="Argentine", num_noticias=15):
    """More reliable news extraction function"""
    #base_url = f"https://news.google.com/rss/search?q={query.replace(' ', '+')}&hl=en-419&gl=AR&ceid=US:es-419"
    base_url = f"https://news.google.com/rss/search?q={query.replace(' ', '+')}&hl=es-419&gl=AR&ceid=AR:es-419"

    try:
        feed = feedparser.parse(base_url)
        if not feed.entries:
            print("No news entries found in the RSS feed")
            return pd.DataFrame()

        noticias = []
        for i, entry in enumerate(feed.entries[:num_noticias]):
            #print(f"Processing {i+1}/{min(len(feed.entries), num_noticias)}: {entry.title[:50]}...")

            content = None
            if hasattr(entry, 'link'):
                content = fetch_article_content(entry.link)
                time.sleep(1)  # Be polite with requests

            if content:
                noticias.append({
                    'titulo': entry.title,
                    'fuente': entry.source.title if hasattr(entry, 'source') else 'Desconocido',
                    'fecha': entry.published if hasattr(entry, 'published') else None,
                    'enlace': entry.link,
                    'contenido': content
                })
            else:
                noticias.append({
                    'titulo': entry.title,
                    'fuente': entry.source.title if hasattr(entry, 'source') else 'Desconocido',
                    'fecha': entry.published if hasattr(entry, 'published') else None,
                    'enlace': entry.link,
                    'contenido': entry.title
                })

        return pd.DataFrame(noticias)

    except Exception as e:
        print(f"Error in news extraction: {e}")
        return pd.DataFrame()



In [58]:
# Extraer noticias (reducido a 30 para demo)
print("Extrayendo noticias con contenido completo...")
df_noticias = extraer_noticias_completas("Argentine", 100)
print(f"\nNoticias extraídas correctamente: {len(df_noticias)}")
#print(f"Ejemplo de contenido:\n{df_noticias['contenido'].iloc[0][:300]}...")

Extrayendo noticias con contenido completo...

Noticias extraídas correctamente: 100


In [59]:
df_noticias.head()

,titulo,fuente,fecha,enlace,contenido
0,Suba de aranceles en EEUU: qué significa la de...,Infobae,"Thu, 03 Apr 2025 03:43:51 GMT",https://news.google.com/rss/articles/CBMi6AFBV...,Suba de aranceles en EEUU: qué significa la de...
1,Qué dicen los documentos de Estados Unidos sob...,Clarin.com,"Thu, 03 Apr 2025 01:16:00 GMT",https://news.google.com/rss/articles/CBMi3AFBV...,Qué dicen los documentos de Estados Unidos sob...
2,¿Cómo afectarán a Argentina los aranceles anun...,El Intransigente,"Thu, 03 Apr 2025 01:47:59 GMT",https://news.google.com/rss/articles/CBMiqAFBV...,¿Cómo afectarán a Argentina los aranceles anun...
3,"Argentina en el abismo: el ""Cisne Negro"" que a...",Ámbito Financiero,"Fri, 04 Apr 2025 12:53:00 GMT",https://news.google.com/rss/articles/CBMitAFBV...,"Argentina en el abismo: el ""Cisne Negro"" que a..."
4,¿Cuántos barcos ingleses hundió la aviación ar...,La Nueva Provincia,"Wed, 02 Apr 2025 18:32:06 GMT",https://news.google.com/rss/articles/CBMiwAFBV...,¿Cuántos barcos ingleses hundió la aviación ar...


In [60]:
def preprocesar_texto(texto):
    # Tokenización y limpieza
    stop_words = set(stopwords.words('spanish') + list(stopwords.words('english')))
    lemmatizer = WordNetLemmatizer()

    # Eliminar URLs y caracteres especiales
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@\w+|\#|\d+', '', texto)

    tokens = word_tokenize(texto.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens
              if token.isalpha() and token not in stop_words and len(token) > 3]

    return tokens

In [62]:
# Aplicar preprocesamiento
print("\nPreprocesando texto...")
df_noticias['tokens'] = df_noticias['contenido'].apply(preprocesar_texto)
df_noticias['texto_limpio'] = df_noticias['tokens'].apply(lambda x: ' '.join(x))


Preprocesando texto...


In [63]:
def modelado_lda(textos, num_topics=5):
    diccionario = Dictionary(textos)
    corpus = [diccionario.doc2bow(texto) for texto in textos]

    modelo = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=42,
        passes=15,
        alpha='auto'
    )

    return modelo, corpus, diccionario

print("\nEntrenando modelo LDA...")
modelo_lda, corpus, diccionario = modelado_lda(df_noticias['tokens'])


Entrenando modelo LDA...


Vamos a usar una visualizaicon interactiva

In [64]:
print("\nGenerando visualización interactiva...")
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(modelo_lda, corpus, diccionario)
vis


Generando visualización interactiva...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.121988  0.040881       1        1  40.410197
1      0.000972 -0.104277       2        1  19.732459
2     -0.067117  0.025892       3        1  19.262009
3     -0.052498  0.061490       4        1  12.366046
0     -0.003346 -0.023986       5        1   8.229290, topic_info=              Term       Freq      Total Category  logprob  loglift
1        aranceles   9.000000   9.000000  Default  30.0000  30.0000
61           milei   7.000000   7.000000  Default  29.0000  29.0000
7          infobae   7.000000   7.000000  Default  28.0000  28.0000
10           trump  10.000000  10.000000  Default  27.0000  27.0000
50       argentino   3.000000   3.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
179          sigue   0.487470   2.211604   Topic5  -4.9733   0.9852
98          españa   0.487470   1.605203   Topic5  -4.9733   1.3057
177       historia   0.487469   1.605204   Topic5  -4.9733   1.3057
247  exportaciones   0.487468   1.599502   Topic5  -4.9733   1.3092
9             suba   0.487468   2.834989   Topic5  -4.9733   0.7369

[259 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
108       1  0.500179     abril
108       2  0.250090     abril
108       4  0.250090     abril
141       1  0.573495  acciones
141       3  0.286748  acciones
...     ...       ...       ...
476       1  0.742999    volvió
31        1  0.794690    ámbito
31        2  0.158938    ámbito
31        4  0.158938    ámbito
107       5  0.717508   últimas

[241 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])